##File Name - Beerwork
#The purpose of this file is to convert data from https://data.opendatasoft.com/explore/dataset/open-beer-database%40public-us/table/
#into a cleaner file

In [121]:
#dependencies and setup - file name "beerwork"
import pandas as pd
import numpy as np
import os
import csv
import xlrd
from sqlalchemy import create_engine

##Try 1 - ParserError

In [122]:
#path to collect data from the resources folder - location on your computer may vary - try 1
#beerdata = "Resources/open-beer-database.csv" 

In [123]:
#loading file into dataframe - note data includes non acii characters - try 1 - failed
#attempt to load csv file into pandas
#beer_db = pd.read_csv(beerdata, encoding= 'unicode_escape')
#beer_db.head()
#ParserError: Error tokenizing data. C error: Expected 3 fields in line 13, saw 10

##Try 2 - Corrupt File

In [124]:
#path to collect data from the resources folder - location on your computer may vary - try 2
#attempt to load Excel file into pandas
#beerdata = "open-beer-database.xls" 

In [125]:
#loading file into dataframe - note data includes non acii characters - try 2 - failed
#beer_db = pd.read_excel(beerdata, encoding= 'unicode_escape')
#beer_db.head()
#XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'<?xml ve'

##Try 3 - Unusable Data

In [126]:
#path to collect data from the resources folder - location on your computer may vary - try 3
#attempt to rename the Excel file into a csv and attempt to load into pandas
#os.rename('open-beer-database.xls', 'open-beer-database.csv') #this only works once
#beer_db = pd.read_csv("open-beer-database.csv", error_bad_lines=False)
#beer_db.head()
#this loads but creates a file unusable data

##Try 4 - ParserError

In [127]:
#path to collect data from the resources folder - location on your computer may vary - try 4
#attempt to rename the Excel file into a csv and attempt to load into pandas
#os.rename('open-beer-database.xls', 'open-beer-database.csv') #this only works once
#beer_db = pd.read_csv("open-beer-database.csv", encoding= 'unicode_escape') 
#ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 43

##Try 5 - This one works!

In [128]:
#path to collect data from the resources folder - location on your computer may vary - try 5
#open csv file first in Excel and then save as a csv uft-8 file - this works
beer_db = pd.read_csv("Resources/open_beer_database_new.csv")

In [129]:
#study data base
beer_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5973 entries, 0 to 5972
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Name                            5963 non-null   object 
 1   id                              5973 non-null   object 
 2   brewery_id                      5963 non-null   object 
 3   cat_id                          5950 non-null   object 
 4   style_id                        5949 non-null   object 
 5   Alcohol By Volume               5948 non-null   float64
 6   International Bitterness Units  5948 non-null   float64
 7   Standard Reference Method       5948 non-null   float64
 8   Universal Product Code          5944 non-null   float64
 9   filepath                        25 non-null     object 
 10  Description                     2046 non-null   object 
 11  add_user                        5930 non-null   object 
 12  last_mod                        59

In [130]:
#study data base
beer_db.head()

,Name,id,brewery_id,cat_id,style_id,Alcohol By Volume,International Bitterness Units,Standard Reference Method,Universal Product Code,filepath,...,last_mod,Style,Category,Brewer,Address,City,State,Country,Coordinates,Website
0,Celis Raspberry,4813,853,5,72,3.9,0.0,0.0,0.0,NaN,...,2010-07-22 16:00:00-04:00,Other Belgian-Style Ales,Belgian and French Ale,Michigan Brewing,1093 Highview Drive,Webberville,Michigan,United States,"42.6616, -84.1946",http://www.michiganbrewing.com/
1,Buck Naked,4864,114,8,96,3.2,0.0,0.0,0.0,NaN,...,2010-07-22 16:00:00-04:00,American-Style Light Lager,North American Lager,Big Buck Brewery,550 South Wisconsin Avenue,Gaylord,Michigan,United States,"45.0223, -84.6826",http://www.bigbuck.com/gaylord.html
2,Fat Belly Amber,93,879,3,33,0.0,0.0,0.0,0.0,NaN,...,2010-07-22 16:00:00-04:00,American-Style Amber/Red Ale,North American Ale,Montana Brewing,113 North Broadway,Billings,Montana,United States,"45.7822, -108.506",NaN
3,Belgian Wit,98,901,-1,-1,0.0,0.0,0.0,0.0,NaN,...,2010-07-22 16:00:00-04:00,NaN,NaN,Nebraska Brewing Company,7474 Towne Center Parkway #101,Papillion,Nebraska,United States,"41.1339, -96.0307",NaN
4,Crystal Ale,172,122,-1,-1,0.0,0.0,0.0,0.0,NaN,...,2010-07-22 16:00:00-04:00,NaN,NaN,Big Sky Brewing,5417 Trumpeter Way,Missoula,Montana,United States,"46.9223, -114.073",NaN


In [131]:
#drop the following columns from the data:
# cat_id, style_id, International Bitterness Units, Standard Reference Method, Universal Product Code, Description, filepath, add_user, last_mod, Website
beer_db.drop(columns=['cat_id','style_id','Alcohol By Volume', 'International Bitterness Units', 'Standard Reference Method', 'Universal Product Code', 'filepath', 'Description', 'add_user', 'last_mod','Coordinates', 'Website'], inplace=True)

In [132]:
#study data base after removal of columns
beer_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5973 entries, 0 to 5972
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        5963 non-null   object
 1   id          5973 non-null   object
 2   brewery_id  5963 non-null   object
 3   Style       4466 non-null   object
 4   Category    4466 non-null   object
 5   Brewer      5948 non-null   object
 6   Address     5191 non-null   object
 7   City        5921 non-null   object
 8   State       5624 non-null   object
 9   Country     5948 non-null   object
dtypes: object(10)
memory usage: 466.8+ KB


In [133]:
#remove NaN values
beer_db = beer_db.dropna(how="any")

In [134]:
#study data base after removal of columns
beer_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3737 entries, 0 to 5972
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        3737 non-null   object
 1   id          3737 non-null   object
 2   brewery_id  3737 non-null   object
 3   Style       3737 non-null   object
 4   Category    3737 non-null   object
 5   Brewer      3737 non-null   object
 6   Address     3737 non-null   object
 7   City        3737 non-null   object
 8   State       3737 non-null   object
 9   Country     3737 non-null   object
dtypes: object(10)
memory usage: 321.1+ KB


In [135]:
beer_db.head()

,Name,id,brewery_id,Style,Category,Brewer,Address,City,State,Country
0,Celis Raspberry,4813,853,Other Belgian-Style Ales,Belgian and French Ale,Michigan Brewing,1093 Highview Drive,Webberville,Michigan,United States
1,Buck Naked,4864,114,American-Style Light Lager,North American Lager,Big Buck Brewery,550 South Wisconsin Avenue,Gaylord,Michigan,United States
2,Fat Belly Amber,93,879,American-Style Amber/Red Ale,North American Ale,Montana Brewing,113 North Broadway,Billings,Montana,United States
9,Log Cabin Nut Brown,1749,754,American-Style Brown Ale,North American Ale,Krogh's Restaurant and Brewpub,23 White Deer Plaza,Sparta,New Jersey,United States
10,Golden Wheat Light,1781,1134,American-Style Lager,North American Lager,Ship Inn Brewpub,61 Bridge Street,Milford,New Jersey,United States


In [136]:
#review country to remove any breweries that are not in the United States
beer_db.Country.unique()

array(['United States', 'Canada', 'Germany', 'Australia',
       'United Kingdom', 'Netherlands', 'Russia', 'India', 'Norway',
       'Belgium', 'Japan', 'Mexico', 'China', 'Ireland', 'Brazil'],
      dtype=object)

In [137]:
#remove all countries that are not the United States
beer_db = beer_db.loc[beer_db["Country"] == "United States", :]
beer_db.Country.unique()


array(['United States'], dtype=object)

In [138]:
#review state names for consistancy
beer_db.State.unique()

array(['Michigan', 'Montana', 'New Jersey', 'Colorado', 'Wisconsin',
       'Vermont', 'California', 'Connecticut', 'New York',
       'New Hampshire', 'Alaska', 'Pennsylvania', 'Illinois',
       'Massachusetts', 'Nevada', 'Washington', 'Arizona', 'Indiana',
       'North Carolina', 'New Mexico', 'Nebraska', 'Rhode Island',
       'Minnesota', 'MA', 'Oregon', 'Utah', 'Louisiana', 'South Carolina',
       'Wyoming', 'Delaware', 'Missouri', 'Maryland', 'Texas', 'Kentucky',
       'District of Columbia', 'Florida', 'NJ', 'Georgia', 'Ohio',
       'Arkansas', 'Hawaii', 'Virginia', 'Tennessee', 'Iowa', 'NC',
       'Oklahoma', 'Maine', 'MO', 'NY', 'Kansas', 'South Dakota', 'KS',
       'Idaho', 'OH', 'WI', 'DC', 'MI'], dtype=object)

In [139]:
#convert abbreviated states to full state names to match the DUI data set
beer_db.replace({'State': {"MA": "Massachusetts", "NJ": "New Jersey", "NC": "North Carolina", "MO": "Missouri", "NY": "New York", "KS": "Kansas", "OH": "Ohio", "WI": "Wisconsin", "DC": "District of Columbia", "MI": "Michigan"}}, inplace=True)

In [140]:
#verify converstion to full state names
beer_db.State.unique()

array(['Michigan', 'Montana', 'New Jersey', 'Colorado', 'Wisconsin',
       'Vermont', 'California', 'Connecticut', 'New York',
       'New Hampshire', 'Alaska', 'Pennsylvania', 'Illinois',
       'Massachusetts', 'Nevada', 'Washington', 'Arizona', 'Indiana',
       'North Carolina', 'New Mexico', 'Nebraska', 'Rhode Island',
       'Minnesota', 'Oregon', 'Utah', 'Louisiana', 'South Carolina',
       'Wyoming', 'Delaware', 'Missouri', 'Maryland', 'Texas', 'Kentucky',
       'District of Columbia', 'Florida', 'Georgia', 'Ohio', 'Arkansas',
       'Hawaii', 'Virginia', 'Tennessee', 'Iowa', 'Oklahoma', 'Maine',
       'Kansas', 'South Dakota', 'Idaho'], dtype=object)

In [141]:
#there isn't any DUI data for DC/District of Columbia so we will delete those records
beer_db = beer_db.loc[beer_db['State'] != "District of Columbia", :]

In [142]:
#verify removal of District of Columbia
beer_db.State.unique()

array(['Michigan', 'Montana', 'New Jersey', 'Colorado', 'Wisconsin',
       'Vermont', 'California', 'Connecticut', 'New York',
       'New Hampshire', 'Alaska', 'Pennsylvania', 'Illinois',
       'Massachusetts', 'Nevada', 'Washington', 'Arizona', 'Indiana',
       'North Carolina', 'New Mexico', 'Nebraska', 'Rhode Island',
       'Minnesota', 'Oregon', 'Utah', 'Louisiana', 'South Carolina',
       'Wyoming', 'Delaware', 'Missouri', 'Maryland', 'Texas', 'Kentucky',
       'Florida', 'Georgia', 'Ohio', 'Arkansas', 'Hawaii', 'Virginia',
       'Tennessee', 'Iowa', 'Oklahoma', 'Maine', 'Kansas', 'South Dakota',
       'Idaho'], dtype=object)

In [143]:
#review file structure one last time before saving
beer_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3190 entries, 0 to 5972
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        3190 non-null   object
 1   id          3190 non-null   object
 2   brewery_id  3190 non-null   object
 3   Style       3190 non-null   object
 4   Category    3190 non-null   object
 5   Brewer      3190 non-null   object
 6   Address     3190 non-null   object
 7   City        3190 non-null   object
 8   State       3190 non-null   object
 9   Country     3190 non-null   object
dtypes: object(10)
memory usage: 274.1+ KB


In [144]:
#save cleaned data to file open_beer_db.csv
beer_db.to_csv("Resources/open_beer_db.csv", encoding='utf-8', index=False)

In [145]:
#load DUI file
dui_db = pd.read_csv("Resources/DUI.csv")

In [146]:
#print head to verify data
dui_db.head()

,State,DUI,Fatalities,Population,Sunday.Sales
0,Alabama,7863,247,4858979,Prohibited
1,Alaska,3163,24,738432,Permitted
2,Arizona,22367,272,6828065,Permitted
3,Arkansas,6919,149,2978204,Local
4,California,141458,914,39144818,Permitted


In [147]:
#rename Sunday.Sales to Sunday_Sales
dui_db.rename(columns={"Sunday.Sales":"Sunday_Sales"})

,State,DUI,Fatalities,Population,Sunday_Sales
0,Alabama,7863,247,4858979,Prohibited
1,Alaska,3163,24,738432,Permitted
2,Arizona,22367,272,6828065,Permitted
3,Arkansas,6919,149,2978204,Local
4,California,141458,914,39144818,Permitted
5,Colorado,25562,151,5456574,Permitted
6,Connecticut,8148,103,3590886,Local
7,Delaware,386,42,945934,Permitted
8,Florida,31783,797,20271272,Local
9,Georgia,19217,365,10214860,Local


In [148]:
#study data base after removal of columns
dui_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   State         50 non-null     object
 1   DUI           50 non-null     int64 
 2   Fatalities    50 non-null     int64 
 3   Population    50 non-null     int64 
 4   Sunday.Sales  50 non-null     object
dtypes: int64(3), object(2)
memory usage: 2.1+ KB


In [149]:
#save cleaned data to file dui_db.csv
dui_db.to_csv("Resources/dui_db.csv", encoding='utf-8', index=False)

In [150]:
#create database connection
connection_string = "postgres:Lunalove2020@localhost:5432/DUIvsBeer"
engine = create_engine(f'postgresql://{connection_string}')

In [151]:
#Confirm tables
engine.table_names()

[]

In [152]:
#Load DataFrames into database
beer_db.to_sql(name='beer_db', con=engine, if_exists='append', index=True)
dui_db.to_sql(name='DUI', con=engine, if_exists='append', index=True)